# Nepal Earthquake - Structural Risk Prediction
__Martin Zagari, Summer Purcschke, and Dave Friesen - ADS-504-01-SU22__

In [1]:
__author__ = 'Martin Zagari, Summer Purschke, Dave Friesen'
__email__ = 'mzagari@sandiego.edu, spurschke@sandiego.edu, dfriesen@sandiego.edu'
__version__ = '1.0'
__date__ = 'August 2022'
__license__ = 'MIT'

### Problem Statement: *[. . .]*  

### Hypothesis: *[. . .]*

## Setup

In [273]:
# Basics
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Modeling
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline, make_pipeline

from sklearn.compose import ColumnTransformer

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report

# Utilities
import joblib
import re
from time import time
import warnings
#warnings.filterwarnings('ignore')
#warnings.resetwarnings()

# Set basic options for consistent output
PRECISION = 2
np.set_printoptions(precision = PRECISION)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.precision', PRECISION)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')

# Set Matplotlib defaults for consistent visualization look 'n' feel
FONTSIZE_S = 10
FONTSIZE_M = 12
FONTSIZE_L = 14
plt.style.use('default')
plt.rcParams['figure.titlesize'] = FONTSIZE_L
plt.rcParams['figure.figsize'] = (7, 7 / (16 / 9))
plt.rcParams['figure.subplot.left'] = '0.1'
plt.rcParams['figure.subplot.bottom'] = '0.1'
plt.rcParams['figure.subplot.top'] = '0.9'
plt.rcParams['figure.subplot.wspace'] = '0.4'
plt.rcParams['lines.linewidth'] = '2'
plt.rcParams['axes.linewidth'] = '2'
plt.rcParams['axes.titlesize'] = '8'
#plt.rcParams['axes.titleweight'] = 'bold'
plt.rcParams['axes.labelsize'] = FONTSIZE_M
plt.rcParams['xtick.labelsize'] = FONTSIZE_S
plt.rcParams['ytick.labelsize'] = FONTSIZE
plt.rcParams['grid.linewidth'] = '1'
plt.rcParams['legend.fontsize'] = FONTSIZE_S
plt.rcParams['legend.title_fontsize'] = FONTSIZE_S

In [105]:
# Set working directory (depending on coder)
%cd '/Users/davidfriesen/Desktop/OneDrive/projects/ADS504_Team_8/data'

/Users/davidfriesen/Library/CloudStorage/OneDrive-Personal/projects/ADS504_Team_8/data


## Data Load, Validation, and Structure Review

In [88]:
# Set row count control totals
structure_ctrl = sum(1 for line in open('csv_building_structure.csv'))
ownership_ctrl = sum(1 for line in open('csv_building_ownership_and_use.csv'))
damage_ctrl = sum(1 for line in open('csv_building_damage_assessment_featex.csv'))

# Read files, accomodating any 'bad' rows
structure_df = pd.read_csv('csv_building_structure.csv', on_bad_lines = 'skip', low_memory = False)
ownership_df = pd.read_csv('csv_building_ownership_and_use.csv', on_bad_lines = 'skip', low_memory = False)
damage_df = pd.read_csv('csv_building_damage_assessment_featex.csv', on_bad_lines = 'skip', low_memory = False)

# Confirm load counts
print('Structure: file=%0d, import=%0d, delta=%0d' %
      (structure_ctrl, len(structure_df), structure_ctrl - len(structure_df)))
print('Ownership: file=%0d, import=%0d, delta=%0d' %
      (ownership_ctrl, len(ownership_df), ownership_ctrl - len(ownership_df)))
print('Damage: file=%0d, import=%0d, delta=%0d' %
      (damage_ctrl, len(damage_df), damage_ctrl - len(damage_df)))

Structure: file=737695, import=737694, delta=1
Ownership: file=762106, import=762105, delta=1
Damage: file=652689, import=652687, delta=2


In [89]:
structure_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 737694 entries, 0 to 737693
Data columns (total 31 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   building_id                             737694 non-null  float64
 1   district_id                             737694 non-null  int64  
 2   vdcmun_id                               737694 non-null  int64  
 3   ward_id                                 737694 non-null  int64  
 4   count_floors_pre_eq                     737694 non-null  int64  
 5   count_floors_post_eq                    737694 non-null  int64  
 6   age_building                            737694 non-null  int64  
 7   plinth_area_sq_ft                       737694 non-null  int64  
 8   height_ft_pre_eq                        737694 non-null  int64  
 9   height_ft_post_eq                       737694 non-null  int64  
 10  land_surface_condition                  7376

In [6]:
ownership_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 762105 entries, 0 to 762104
Data columns (total 17 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   building_id                    762105 non-null  float64
 1   district_id                    762105 non-null  int64  
 2   vdcmun_id                      762105 non-null  int64  
 3   ward_id                        762105 non-null  int64  
 4   legal_ownership_status         762105 non-null  object 
 5   count_families                 762103 non-null  float64
 6   has_secondary_use              762095 non-null  float64
 7   has_secondary_use_agriculture  762105 non-null  int64  
 8   has_secondary_use_hotel        762105 non-null  int64  
 9   has_secondary_use_rental       762105 non-null  int64  
 10  has_secondary_use_institution  762105 non-null  int64  
 11  has_secondary_use_school       762105 non-null  int64  
 12  has_secondary_use_industry    

In [7]:
damage_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 652687 entries, 0 to 652686
Data columns (total 17 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   building_id                            652687 non-null  float64
 1   district_id                            652687 non-null  int64  
 2   vdcmun_id                              652687 non-null  int64  
 3   ward_id                                652687 non-null  int64  
 4   damage_overall_collapse                436039 non-null  object 
 5   damage_overall_leaning                 436038 non-null  object 
 6   area_assesed                           652676 non-null  object 
 7   damage_grade                           652676 non-null  object 
 8   technical_solution_proposed            652676 non-null  object 
 9   has_geotechnical_risk                  652676 non-null  float64
 10  has_geotechnical_risk_land_settlement  652687 non-null  

>### Preliminary Dataset Prep and Feature Reduction

In [8]:
# Eliminate features considered n/a to problem statement and hypothesis
structure_dr = structure_df.drop(
    columns = ['district_id', 'vdcmun_id', 'ward_id',
               'height_ft_post_eq', 'condition_post_eq', 'technical_solution_proposed']
)
ownership_dr = ownership_df.drop(
    columns = ['district_id', 'vdcmun_id', 'ward_id']
)
damage_dr = damage_df.drop(
    columns=['district_id', 'vdcmun_id', 'ward_id', 'damage_grade']
)

# Merge all three dataframes into common set of label and prospective features
temp_df = pd.merge(structure_dr, ownership_dr, how = 'inner', on = 'building_id')
building_df = pd.merge(temp_df, damage_dr, how = 'inner', on = 'building_id')
building_df.drop('building_id', axis = 1, inplace = True)
building_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 652686 entries, 0 to 652685
Data columns (total 49 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   count_floors_pre_eq                     652686 non-null  int64  
 1   count_floors_post_eq                    652686 non-null  int64  
 2   age_building                            652686 non-null  int64  
 3   plinth_area_sq_ft                       652686 non-null  int64  
 4   height_ft_pre_eq                        652686 non-null  int64  
 5   land_surface_condition                  652686 non-null  object 
 6   foundation_type                         652686 non-null  object 
 7   roof_type                               652686 non-null  object 
 8   ground_floor_type                       652686 non-null  object 
 9   other_floor_type                        652686 non-null  object 
 10  position                                6526

>### Class Label Analysis and Preparation

In [9]:
# Identify target (y)
target_col = 'damage_grade'

# Summarize target class balance
label_count = len(building_df)
label_props = (building_df.groupby(target_col, dropna = False)[target_col].count() / label_count * 100).sort_values(ascending = False)
print('Target class (label) proportions:'); print(label_props.to_string(header = False))

Target class (label) proportions:
Grade 5   35.19
Grade 4   25.15
Grade 3   18.35
Grade 2   11.29
Grade 1   10.02
NaN        0.00


In [10]:
# Drop (relatively immaterial number of) null labels
print('Starting rows:', len(building_df), end = ' ')
building_df.dropna(axis = 0, subset = target_col, inplace = True)
print('Ending rows:', len(building_df))

target_classes = sorted(building_df[target_col].unique())

Starting rows: 652686 Ending rows: 652675


>### Data Types and Attribute Names (confirm/update)

In [11]:
# Identify prospective feature columns by 'type'
num_cols = ['age_building',
            'plinth_area_sq_ft',
            'height_ft_pre_eq',
            'count_families']
cat_cols = ['land_surface_condition',
            'foundation_type',
            'roof_type',
            'ground_floor_type',
            'other_floor_type',
            'position',
            'plan_configuration',
            'legal_ownership_status',
            'damage_overall_collapse',
            'damage_overall_leaning',
            'area_assesed',
            'technical_solution_proposed']
bin_cols = ['has_superstructure_adobe_mud',
            'has_superstructure_mud_mortar_stone',
            'has_superstructure_stone_flag',
            'has_superstructure_cement_mortar_stone',
            'has_superstructure_mud_mortar_brick',
            'has_superstructure_cement_mortar_brick',
            'has_superstructure_timber',
            'has_superstructure_bamboo',
            'has_superstructure_rc_non_engineered',
            'has_superstructure_rc_engineered',
            'has_superstructure_other',
            'has_secondary_use',
            'has_secondary_use_agriculture',
            'has_secondary_use_hotel',
            'has_secondary_use_rental',
            'has_secondary_use_institution',
            'has_secondary_use_school',
            'has_secondary_use_industry',
            'has_secondary_use_health_post',
            'has_secondary_use_gov_office',
            'has_secondary_use_use_police',
            'has_secondary_use_other',
            'has_geotechnical_risk',
            'has_geotechnical_risk_land_settlement',
            'has_geotechnical_risk_fault_crack',
            'has_geotechnical_risk_liquefaction',
            'has_geotechnical_risk_landslide',
            'has_geotechnical_risk_rock_fall',
            'has_geotechnical_risk_flood',
            'has_geotechnical_risk_other']

In [12]:
# Confirm [uniform] types
print(building_df.dtypes[num_cols].to_string())
print(building_df.dtypes[cat_cols].to_string())
print(building_df.dtypes[bin_cols].to_string())

age_building           int64
plinth_area_sq_ft      int64
height_ft_pre_eq       int64
count_families       float64
land_surface_condition         object
foundation_type                object
roof_type                      object
ground_floor_type              object
other_floor_type               object
position                       object
plan_configuration             object
legal_ownership_status         object
damage_overall_collapse        object
damage_overall_leaning         object
area_assesed                   object
technical_solution_proposed    object
has_superstructure_adobe_mud                int64
has_superstructure_mud_mortar_stone         int64
has_superstructure_stone_flag               int64
has_superstructure_cement_mortar_stone      int64
has_superstructure_mud_mortar_brick         int64
has_superstructure_cement_mortar_brick      int64
has_superstructure_timber                   int64
has_superstructure_bamboo                   int64
has_superstructure_rc_non_en

In [13]:
# Confirm column contents. . .

In [14]:
# Update types as needed (for consistency)
building_df['count_families'] = building_df['count_families'].astype(int)
building_df['has_superstructure_other'] = building_df['has_superstructure_other'].astype(int)
building_df['has_secondary_use'] = building_df['has_secondary_use'].astype(int)
building_df['has_geotechnical_risk'] = building_df['has_geotechnical_risk'].astype(int)

## Data Partitioning

In [15]:
# Segregate data into predictor (X) and target (y) dataframes
building_X = building_df.loc[:, building_df.columns != target_col].copy()
building_y = building_df[target_col]

# Partition data 70/30, stratifying for class balance (ref. above)
X_train, X_test, y_train, y_test = train_test_split(
    building_X, building_y, test_size = 0.3, random_state = 42, stratify = building_y
)

## Univariate Analysis and Feature Engineering

>### Missing/Null Values (find/impute/drop)

In [16]:
na_df = pd.DataFrame(X_train[num_cols + cat_cols + bin_cols].isna().sum()) / len(X_train) * 100
na_df.index.name = 'column'; na_df.reset_index(inplace = True)
print('Proportion of nulls:'); print(na_df[na_df[0] > 0].to_string(index = False, header = False))

Proportion of nulls:
damage_overall_collapse 33.19
 damage_overall_leaning 33.19


In [17]:
# Address missing/null values here? Or, pipeline? For both train, test. . .
X_train['damage_overall_collapse'].fillna('[missing]', inplace = True)
X_test['damage_overall_collapse'].fillna('[missing]', inplace = True)

X_train['damage_overall_leaning'].fillna('[missing]', inplace = True)
X_test['damage_overall_leaning'].fillna('[missing]', inplace = True)

>### Categorical Features (encoding)

In [18]:
cat_count = len(X_train)
for cc in cat_cols:
    print()
    print(cc, '-')
    for cv in pd.unique(X_train[cc]):
        if isinstance(cv, float) != True:
            print(cv, '(%.2f)' % (X_train[X_train[cc] == cv][cc].count() / cat_count * 100))


land_surface_condition -
Flat (83.32)
Steep slope (3.10)
Moderate slope (13.58)

foundation_type -
Mud mortar-Stone/Brick (85.61)
Bamboo/Timber (4.70)
RC (4.01)
Cement-Stone/Brick (5.10)
Other (0.58)

roof_type -
Bamboo/Timber-Light roof (68.50)
Bamboo/Timber-Heavy roof (25.74)
RCC/RB/RBC (5.76)

ground_floor_type -
Mud (81.04)
Brick/Stone (9.12)
RC (9.23)
Timber (0.46)
Other (0.15)

other_floor_type -
Not applicable (15.59)
Timber-Planck (15.56)
TImber/Bamboo-Mud (64.66)
RCC/RB/RBC (4.19)

position -
Not attached (79.07)
Attached-1 side (17.35)
Attached-2 side (3.43)
Attached-3 side (0.15)

plan_configuration -
Rectangular (96.16)
Square (2.37)
T-shape (0.11)
L-shape (1.10)
Building with Central Courtyard (0.01)
U-shape (0.05)
Others (0.05)
Multi-projected (0.13)
H-shape (0.01)
E-shape (0.02)

legal_ownership_status -
Private (96.74)
Public (1.84)
Institutional (1.10)
Other (0.33)

damage_overall_collapse -
[missing] (33.19)
Moderate-Heavy (27.74)
None (9.12)
Severe-Extreme (16.07)
I

In [19]:
# Encode categorical label (y)
label_enc = LabelEncoder().fit(y_train)
y_train = label_enc.transform(y_train)
y_test = label_enc.transform(y_test)

# Encode other here? Or, pipeline (currently handled in pipeline)?

>### Outliers (convert/drop)

In [20]:
print('Skewness:'); print(pd.DataFrame(building_X[num_cols].skew()).to_string(header = False))

Skewness:
age_building      13.52
plinth_area_sq_ft  3.80
height_ft_pre_eq   1.28
count_families     1.51


In [21]:
# Address outliers here?

>### Centering/Scaling (standardizing/normalizing)

In [22]:
# Address centering/scaling here? Or, pipeline (currently handled in pipeline)?

>### "Bad"/Duplicate Data (find/convert/drop)

In [23]:
print(building_X[bin_cols].max() - building_X[bin_cols].min())

has_superstructure_adobe_mud              1
has_superstructure_mud_mortar_stone       1
has_superstructure_stone_flag             1
has_superstructure_cement_mortar_stone    1
has_superstructure_mud_mortar_brick       1
has_superstructure_cement_mortar_brick    1
has_superstructure_timber                 1
has_superstructure_bamboo                 1
has_superstructure_rc_non_engineered      1
has_superstructure_rc_engineered          1
has_superstructure_other                  1
has_secondary_use                         1
has_secondary_use_agriculture             1
has_secondary_use_hotel                   1
has_secondary_use_rental                  1
has_secondary_use_institution             1
has_secondary_use_school                  1
has_secondary_use_industry                1
has_secondary_use_health_post             1
has_secondary_use_gov_office              1
has_secondary_use_use_police              1
has_secondary_use_other                   1
has_geotechnical_risk           

## Multivariate Analysis, Engineering, Feature Selection

In [24]:
#
corr_df = X_train[num_cols].corr()
corr_df.rename(columns = lambda s: s[0:19], index = lambda s: s[0:19], inplace = True)
print(corr_df)

#
# Do something like Cramer's V for categorical correlation?

                   age_building  plinth_area_sq_ft  height_ft_pre_eq  count_families
age_building           1.00           -0.01               0.04             0.00     
plinth_area_sq_ft     -0.01            1.00               0.22             0.11     
height_ft_pre_eq       0.04            0.22               1.00             0.07     
count_families         0.00            0.11               0.07             1.00     


## Modeling (iteration n)

In [310]:
# Standardized modeling function, to create and execute a modeling pipeline through
#   to model performance metrics
def model(algorithm, iteration, params, classes, X_tr, y_tr, X_te, y_te, cv, plot_cm):
    # Create transformers
    cat_encoder = OneHotEncoder(handle_unknown = 'ignore', sparse = False)
    standardizer = StandardScaler()
    
    # Create preprocessor
    cat_transformer = make_pipeline(cat_encoder)
    num_transformer = make_pipeline(standardizer)
    preprocessor = ColumnTransformer(
        [('cat', cat_transformer, cat_cols),
         ('num', num_transformer, num_cols)]
    )

    # Prepare algorithm with hyperparameters+
    algorithm.set_params(**params)
    algorithm_name = type(algorithm).__name__ + ' (iteration ' + str(iteration) + ')'
    
    # Create pipeline and cross-validate model (for later output)
    pipe = make_pipeline(preprocessor, algorithm)
    
    # Fit model
    print('\n\n', 'Fitting ' + algorithm_name + '...', end = ' ')
    fit_start = time()
    pipe.fit(X_tr, y_tr)
    fit_time = time() - fit_start
    print('done in %0.3fs.' % fit_time)
    
    # Cross-validate model
    scores = []
    cv_time = 0
    if (cv > 0):
        print('Cross-validating ' + algorithm_name + '...', end = ' ')
        cv_start = time()
        scores = cross_val_score(pipe, X_tr, y_tr, cv = cv, scoring = 'accuracy')
        cv_time = time() - cv_start
        print('done in %0.3fs.' % cv_time)
        print(scores)
    
    # Validate model
    print('Validating ' + algorithm_name + '...', end = ' ')
    val_start = time()
    y_tr_pred = pipe.predict(X_tr)
    y_te_pred = pipe.predict(X_te)
    val_time = time() - val_start
    print('done in %0.2fs.' % val_time)

    # Show validation results
    if plot_cm:
        fig, ax = plt.subplots()
        cmd = ConfusionMatrixDisplay(confusion_matrix(y_te, y_te_pred), display_labels = classes)
        cmd.plot(ax = ax)
        plt.suptitle(algorithm_name, y = 1)
        plt.title(params)
        plt.show()

    print('\n', algorithm_name)
    print(params)
    print(classification_report(y_te, y_te_pred))    
    
    # Persist pipeline (+model)
    pipe_filename = algorithm_name + '.pipe'
    joblib.dump(pipe, pipe_filename)
    
    # Persist results
    results = []
    results.append ({
        'algorithm': algorithm_name,
        'parameters': params,
        'fit_time': fit_time,
        'cv_time': cv_time,
        'val_time': val_time,
        'scores': tuple(scores),
        'train_acc': accuracy_score(y_tr_pred, y_tr),
        'test_acc': accuracy_score(y_te_pred, y_te)
    })
    results_filename = algorithm_name + '.results'
    joblib.dump(results, results_filename)

    return pd.DataFrame(results)

In [315]:
# Iterate on models (preliminary)
clf_results = pd.DataFrame()

#
for depth in range(5, 11):
    tree_r = model(DecisionTreeClassifier(), (depth - 4),
                   {'max_depth': depth},
                   target_classes, X_train, y_train, X_test, y_test,
                   cv = 0, plot_cm = False)
    clf_results = pd.concat([clf_results, tree_r])

#
perc_r = model(Perceptron(), 1,
               {'class_weight': 'balanced'},
               target_classes, X_train, y_train, X_test, y_test,
               cv = 0, plot_cm = False)
clf_results = pd.concat([clf_results, perc_r])

for i in range(1, 3):
    knn_r = model(KNeighborsClassifier(), i,
                  {'n_neighbors': 2 * i + 1},
                  target_classes, X_train, y_train, X_test, y_test,
                  cv = 0, plot_cm = False)
    clf_results = pd.concat([clf_results, knn_r])

logr_r = model(LogisticRegression(), 1,
               {'solver': 'saga', 'max_iter': 400, 'multi_class': 'multinomial'},
               target_classes, X_train, y_train, X_test, y_test,
               cv = 0, plot_cm = False)
clf_results = pd.concat([clf_results, logr_r])

svm_r = model(LinearSVC(), 1,
              {'multi_class': 'crammer_singer', 'max_iter': 2000},
              target_classes, X_train, y_train, X_test, y_test,
              cv = 0, plot_cm = False)
clf_results = pd.concat([clf_results, svm_r])

clf_results



 Fitting DecisionTreeClassifier (iteration 1)... done in 1.698s.
Validating DecisionTreeClassifier (iteration 1)... done in 1.09s.

 DecisionTreeClassifier (iteration 1)
{'max_depth': 5}
              precision    recall  f1-score   support

           0       0.99      0.63      0.77     19623
           1       0.66      0.84      0.74     22106
           2       0.79      0.73      0.76     35922
           3       0.79      0.91      0.85     49252
           4       1.00      0.95      0.97     68900

    accuracy                           0.85    195803
   macro avg       0.85      0.81      0.82    195803
weighted avg       0.87      0.85      0.86    195803



 Fitting DecisionTreeClassifier (iteration 2)... done in 1.825s.
Validating DecisionTreeClassifier (iteration 2)... done in 1.10s.

 DecisionTreeClassifier (iteration 2)
{'max_depth': 6}
              precision    recall  f1-score   support

           0       0.99      0.63      0.77     19623
           1       0.66 

,algorithm,parameters,fit_time,cv_time,val_time,scores,train_acc,test_acc
0,DecisionTreeClassifier (iteration 1),{'max_depth': 5},1.70,0,1.09,(),0.85,0.85
0,DecisionTreeClassifier (iteration 2),{'max_depth': 6},1.82,0,1.10,(),0.85,0.85
0,DecisionTreeClassifier (iteration 3),{'max_depth': 7},1.92,0,1.12,(),0.86,0.86
0,DecisionTreeClassifier (iteration 4),{'max_depth': 8},2.05,0,1.13,(),0.86,0.86
0,DecisionTreeClassifier (iteration 5),{'max_depth': 9},2.17,0,1.14,(),0.86,0.86
0,DecisionTreeClassifier (iteration 6),{'max_depth': 10},2.18,0,1.10,(),0.86,0.86
0,Perceptron (iteration 1),{'class_weight': 'balanced'},5.63,0,1.09,(),0.82,0.82
0,KNeighborsClassifier (iteration 1),{'n_neighbors': 3},0.85,0,333.99,(),0.90,0.83
0,KNeighborsClassifier (iteration 2),{'n_neighbors': 5},0.84,0,354.24,(),0.88,0.84
0,LogisticRegression (iteration 1),"{'solver': 'saga', 'max_iter': 400, 'multi_cla...",225.12,0,1.10,(),0.86,0.86
